# Extraction des données IIT-CDIP

## README
Ce notebook permet de télécharger sur le site "https://data.nist.gov" les images et métadonnées de la BDD IIT-CDIP associées aux images RVL-CDIP.

Il réalise tout d'abord certaines opérations préalables (chapitre 1), dont la configuration des répertoires du projet


A l'issue (chapitre 2), il reconstitue la liste des identifiants de documents de la BDD RVL-CDIP qui se trouvent dans le répertoire *rvl_cdip_images_path*.

Ensuite (chapitre 3), il identifie les images tar à télécharger (en ignorant celles qui ont déjà été pleinement exploitées), les télécharge sur le site *data.nist.gov* puis écrit sur le disque dur, dans l'arborescence du répertoire *iit_cdip_images_path*, les fichiers tif et xml concernés.

Remarques:
- Lors de cette phase, les fichiers .tar téléchargés sont volumineux (environ 2.5 GB par fichier, et environ 600 fichiers tar au total, ce qui représente plus d'1 To de données à télécharger). Toutefois, les fichiers à conserver ne réprésentent qu'un peu moins de 70Go. Pour éviter des écritures disque inutiles et l'occupation d'un très gros volume, il a été choisi de traiter les fichiers téléchargés à la volée, sans les écrire sur le disque dur. L'inconvénient est que si l'on souhaite accéder à nouveau à ce fichier tar, il faudra le télécharger à nouveau. L'avantage, au delà de l'économie d'usure des disques durs) est de limiter l'espace disque requis.
- Dans les fichiers .tar téléchargés, certains documents ne disposent pas de fichiers xml associés. Dans ce cas, seule l'image est obtenue.

Enfin (chapitre 4), le notebook permet de télécharger les fichiers xml "collectifs", dans lesquels sont rassemblées les métadonnées de plusieurs documents (environ 10000 par fichier). Les fichiers téléchargés sont traités à la volée pour être extraits et réduits afin de ne conserver que les métadonnées liées aux documents de la base de données RVL_CDIP. Ce traitement évite à nouveaux des accès disque et permet de faire passer le volume requis d'environ 24 Go à environ 3 Go.

Remarque:
- **[IMPORTANT] Au bilan, l'utilisation de ce script écrira environ 70 Go de fichiers tif et xml sur le disque dur et téléchargera environ 1,5 To d'internet**.


## 1. Préparation

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import os
import io
import gzip
import time
import requests
import tarfile
import pandas as pd
from lxml import etree

## 2. Création de la liste des identifiants des images RVL-CDIP

In [ ]:
def get_rvl_cdip_image_ids(rvl_cdip_images_path):
    tmp_list = []
    for foldername, _, filenames in os.walk(rvl_cdip_images_path):
        if filenames:
            filename = filenames[0]
            # we check that the structure is relevant to our expectation with 2 asserts
            assert len(filenames) == 1, f"{foldername},{filename}"
            if filename.startswith('.'): # avoid to consider files like .DS_Store on mac
                continue
            assert filename.endswith(".tif"), f"{foldername},{filename}"
            tmp_list.append((os.path.basename(foldername), filename))
    tmp_list.sort()
    return pd.DataFrame(tmp_list, columns = ["document_id", "filename"])


In [ ]:
t = time.time()
df = get_rvl_cdip_image_ids(PATHS.rvl_cdip_images)
print(f"Duree d'exécution: {time.time() - t:.3f} secondes.")
print(f"({len(df)} images traitées)\n")

df.head()

In [ ]:
df.loc[:, "first_letter"] = df.document_id.str.slice(stop = 1)
df.loc[:, "first_two_letters"] = df.document_id.str.slice(stop = 2)
df.head()

## 3. Téléchargement des images et fichiers xml individuels de la base IIT-CDIP

In [ ]:
groupby_images = df.groupby("first_two_letters").agg(
    {'document_id': list, 'filename': list}).rename(
    columns={"document_id": "document_ids", "filename":"filenames"})
groupby_images.head()

In [ ]:
def remove_existing_ids(ids_list: list, filenames_list: list, iit_cdip_images_path: str, also_check_xmls = True):
    """From given lists of image ids and filenames, remove those for which image already exists in iit_cdip_images_path
    Return a new list containing ids that do not exist yet.
    """
    missing_ids = []
    missing_filenames = []
    for id_, filename in zip(ids_list, filenames_list):
        expected_file_path = PATHS.iit_cdip_images / f"images{id_[0]}" / id_[0] / id_[1] / id_[2] / id_ / filename
        # expected_file_path = "/Users/ben/Work/mle/ds-project/mai25_bds_extraction/data_sample/raw/IIT-CDIP/images/imagesa/a/a/a/aaa0a000/92464841_4842.tif"
        image_exists = expected_file_path.exists()
        
        if also_check_xmls:
            expected_file_path = PATHS.iit_cdip_images / f"images{id_[0]}" / id_[0] / id_[1] / id_[2] / id_ / f"{id_}.xml"
            xml_exists = expected_file_path.exists()
        else:
            xml_exists = True
        if not (image_exists and xml_exists):
            missing_ids.append(id_)
            missing_filenames.append(filename)
    return missing_ids, missing_filenames

In [ ]:
nist_images_base_url = "https://data.nist.gov/od/ds/ark:/88434/mds2-2531/cdip-images/"
def import_iit_cdip_images_set(letters, ids, filenames, iit_cdip_images_path, iit_cdip_xmls_path, nist_images_base_url):
    missing_files = [
        os.path.join(f"images{id_[0]}", id_[0], id_[1], id_[2], id_, filename) 
                    for id_, filename in zip(ids, filenames)] + [
        os.path.join(f"images{id_[0]}", id_[0], id_[1], id_[2], id_, f"{id_}.xml") 
                    for id_ in ids]
    # exemple d'url d'un dossier d'images sur NIST: 
    # https://data.nist.gov/od/ds/ark:/88434/mds2-2531/cdip-images/images.a.a.tar
    url = f"{nist_images_base_url}images.{letters[0]}.{letters[1]}.tar"
    print("   ... fetching ", url)
    t = time.time()
    tif_amount, xml_amount = 0, 0
    reponse = requests.get(url, stream=True)
    reponse.raise_for_status()  # En cas d'erreur HTTP
    with tarfile.open(fileobj=io.BytesIO(reponse.content), mode="r|*") as archive:
        for file in archive:
            if file.name in missing_files:
                archive.extract(file, path=PATHS.iit_cdip_images)
                if file.name.endswith('.tif'):
                    tif_amount += 1
                else:
                    xml_amount += 1
    print(f"   ... Successfully extracted {tif_amount} tif + {xml_amount} xml files, in {time.time() - t:.2f} seconds")

In [ ]:
for index, serie in groupby_images.iterrows():
    letters = index
    ids_list = serie.document_ids
    filenames_list = serie.filenames
    missing_ids, missing_filenames = remove_existing_ids(ids_list, filenames_list, PATHS.iit_cdip_images, also_check_xmls = False)
    print(f"Processing letters {letters}: {len(missing_ids)} images to retrieve")
    if missing_ids:
        import_iit_cdip_images_set(letters, missing_ids, missing_filenames, PATHS.iit_cdip_images, PATHS.iit_cdip_xmls, nist_images_base_url)


## 4. Téléchargement des xmls communs 

In [ ]:
# urls de téléchargement=
# https://data.nist.gov/od/ds/ark:/88434/mds2-2531/cdip-text/cdip-1.tar
# de 1 à 6

for n in range(1, 7):
    print(f"Downloading  du fichier {n}/6")
    url = f"https://data.nist.gov/od/ds/ark:/88434/mds2-2531/cdip-text/cdip-{n}.tar"
    reponse = requests.get(url, stream=True)
    with tarfile.open(fileobj=io.BytesIO(reponse.content), mode="r|*") as archive:
        for gz_info in archive:
            letters = gz_info.name[8] + gz_info.name[10]
            rvl_ids = rvl_cdip_df[rvl_cdip_df.document_id.str.startswith(letters)].document_id.tolist()
            output_filename = PATHS.iit_cdip_xmls / f"{letters}.xml"
            z_file = archive.extractfile(gz_info)
            compressed_data = z_file.read()
            decompressed_data = gzip.decompress(compressed_data)
            try:
                xml_file = io.BytesIO(decompressed_data)
                parser = etree.XMLParser(recover=True)
                tree = etree.parse(xml_file, parser)
                root = tree.getroot()
            except Exception as e:
                print(f"Erreur de parsing XML : {e}")
                continue    
            print(gz_info.name, letters, ":", "existing records=", len(root.findall("record")), end="")
            for record in root.findall("record"):
                docid_text = getattr(record.find("docid"), "text", None)
                if docid_text not in rvl_ids:
                    root.remove(record)
            tree.write(output_filename)
            print(" kept ", len(root.findall("record")), " out of ", len(rvl_ids), " possibles.")
                
                